In [1]:
from src.criterion import right_censored
from src.load_data import load_datasets,load_dataframe
from src.utils import train

from torch.optim import Adam
import torch
from torch.utils.data import DataLoader
import torch.optim as optim

import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm
import pandas as pd
import numpy as np

import random
import time

C:\Users\lpott\anaconda3\envs\survival\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\lpott\anaconda3\envs\survival\lib\site-packages\torch\torch_version.py:21: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import packaging  # type: ignore[attr-defined]


In [2]:
class ARGS(object):
    def __init__(self):
        pass

args = ARGS()
args.verify=False
args.device="cpu"

args.seed = 123

args.eps=0.5
args.norm=np.inf
args.bound_type = "CROWN-IBP"
args.num_epochs=100
args.lr = 5e-4
args.batch_size= 512
args.scheduler_name = "SmoothedScheduler"
args.scheduler_opts = "start=1000,length=20"
args.hidden_dims = [15,15]
args.save_model = ""
args.dataset = "Dialysis"

In [3]:
# GOOD DATASETS
# 1. TRACE
# 2. divorce 
# 3. Dialysis
dataset_train,dataset_test = load_datasets(args.dataset,test_size=0.2)

In [4]:
dataloader_train = DataLoader(dataset_train,batch_size=args.batch_size,shuffle=True)
dataloader_test = DataLoader(dataset_test,batch_size=args.batch_size,shuffle=False)

dataloader_train.mean = dataloader_test.mean = dataset_train.mean
dataloader_train.std = dataloader_test.std = dataset_train.std


_,args.input_dims = dataset_train.tensors[0].shape
args.output_dims = 1

In [5]:
import torch.nn as nn

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F


In [90]:
class Weibull_Model(nn.Module):
    def __init__(self,input_dim,output_dim=1):
        super().__init__()

        self.linear1 = nn.Linear(input_dim,10)
        self.linear2 = nn.Linear(10,1)

    def forward(self,x):
        
        x = self.linear1(x)
        x = F.relu(x)
        x = self.linear2(x)
        
        return torch.exp(x)

In [91]:
clf = Weibull_Model(input_dim=args.input_dims,output_dim=args.output_dims)

optimizer = Adam(clf.parameters(),lr=args.lr)

In [92]:
# class RightCensorWrapper(nn.Module):
#     def __init__(self,model):
#         super(RightCensorWrapper,self).__init__()
#         self.model = model

#     def forward(self,x,t,e):
#         parameters = self.model(x)
        
#         # rate = torch.matmul(parameters,torch.FloatTensor([[1],[0]]))
#         # k = torch.matmul(parameters,torch.FloatTensor([[0],[1]]))


#         # print(k.shape)
#         # print(rate.shape)

#         # log_exact = e * torch.log(rate) + e*torch.log(k) + e * (k-1)*torch.log(rate) + e*(k-1)*torch.log(t) + e * -(torch.pow(t,k) * torch.pow(rate,k))
#         # log_right = (1 - e) * -(torch.pow(rate,k) * torch.pow(t,k))
#         # log_right = (1 - e) * parameters #-(torch.pow(parameters,2.0) * torch.pow(t,2.0))
#         parameters = parameters*e+t
#         return torch.mean(parameters,axis=1,keepdims=True)

In [136]:
class RightCensorWrapper(nn.Module):
    def __init__(self,model):
        super(RightCensorWrapper,self).__init__()
        self.model = model

    def forward(self,x,t,e):
        rate = self.model(x)

        log_exact = e * torch.log(rate) + e * -(t * rate)
        log_right = (1 - e) * -(rate * t)

        return -log_exact + -log_right

In [137]:
def right_censored(rate,k, t,e):
    log_exact = e * torch.log(rate) * e*torch.log(k) + e * (k-1)*torch.log(rate) + e*(k-1)*torch.log(t) + e * -(torch.pow(t,k) * torch.pow(rate,k))
    log_right = (1 - e) * - (torch.pow(rate,k) * torch.pow(t,k))

    return (-log_exact + -log_right).sum()

In [138]:
from auto_LiRPA import BoundedModule, BoundedTensor
from auto_LiRPA.perturbations import *
from auto_LiRPA.utils import MultiAverageMeter
from auto_LiRPA.eps_scheduler import LinearScheduler, AdaptiveScheduler, SmoothedScheduler, FixedScheduler

In [139]:
X_train,T_train,E_train = dataset_train.tensors

In [165]:
torch.pow(clf(X_train),clf(X_train))

tensor([[0.9068],
        [0.9417],
        [1.0010],
        ...,
        [1.0702],
        [0.9447],
        [1.1344]], grad_fn=<PowBackward1>)

In [184]:
class RightCensorWrapper(nn.Module):
    def __init__(self,model):
        super(RightCensorWrapper,self).__init__()
        self.model = model

    def forward(self,x,t,e):
        rate = self.model(x)[:,[0]]
        k = self.model(x)[:,[1]]

        log_exact = e * torch.log(rate) + e * -(t * rate)
        log_right = torch.pow(rate,k)

        return -log_exact + -log_right
        
class Weibull_Model(nn.Module):
    def __init__(self,input_dim,output_dim=1):
        super().__init__()

        self.linear1 = nn.Linear(input_dim,10)
        self.linear2 = nn.Linear(10,2)

    def forward(self,x):
        
        x = self.linear1(x)
        x = F.relu(x)
        x = self.linear2(x)
        
        return torch.exp(x)
epsilon = 0.1 
from auto_LiRPA import BoundedModule, BoundedTensor, PerturbationLpNorm
model = Weibull_Model(X_train.shape[1],1)


my_input = X_train
# Wrap the model with auto_LiRPA.
model = BoundedModule(RightCensorWrapper(model), dataloader_train.dataset.tensors)
# Define perturbation. Here we add Linf perturbation to input data.
ptb = PerturbationLpNorm(norm=np.inf, eps=0.1)
# Make the input a BoundedTensor with the pre-defined perturbation.
my_input = BoundedTensor(X_train, ptb)
# # Regular forward propagation using BoundedTensor works as usual.
# prediction = model(my_input)
# Compute LiRPA bounds using the backward mode bound propagation (CROWN).
lb, ub = model.compute_bounds(x=(my_input,T_train,E_train), method="backward",IBP=True)
ub

AssertionError: 

In [128]:
from auto_LiRPA import BoundedModule, BoundedTensor, PerturbationLpNorm

# Define computation as a nn.Module.
class MyModel(nn.Module):
    def __init__(self,input_dim,output_dim=1):
        super().__init__()

        self.linear1 = nn.Linear(input_dim,10)
        self.linear2 = nn.Linear(10,1)

    def forward(self,x):
        
        x = self.linear1(x)
        x = F.relu(x)
        x = self.linear2(x)
        
        return torch.exp(x)
model = MyModel(X_train.shape[1],1)

my_input = X_train
# Wrap the model with auto_LiRPA.
model = BoundedModule(model, my_input)
# Define perturbation. Here we add Linf perturbation to input data.
ptb = PerturbationLpNorm(norm=np.inf, eps=0.1)
# Make the input a BoundedTensor with the pre-defined perturbation.
my_input = BoundedTensor(X_train, ptb)
# # Regular forward propagation using BoundedTensor works as usual.
# prediction = model(my_input)
# Compute LiRPA bounds using the backward mode bound propagation (CROWN).
lb, ub = model.compute_bounds(x=(my_input,), method="backward",IBP=False)

tensor([[1.0168],
        [0.9859],
        [1.0457],
        ...,
        [0.9438],
        [0.9845],
        [1.0355]], grad_fn=<ViewBackward0>)

In [ ]:
model_robust = BoundedModule(RightWeibullCensorWrapper(clf), (X_train,T_train,E_train))
epsilon = 1e-4
# Define perturbation. Here we add Linf perturbation to input data.
ptb = PerturbationLpNorm(norm=np.inf, eps=torch.Tensor([epsilon]))
# Make the input a BoundedTensor with the pre-defined perturbation.
my_input = BoundedTensor(torch.Tensor(X_train), ptb)
# Regular forward propagation using BoundedTensor works as usual.
# prediction = model(my_input,torch.rand_like(T_train),torch.rand_like(E_train))
# Compute LiRPA bounds using CROWN
lb, ub = model_robust.compute_bounds(method="backward",x=(my_input))

In [ ]:
def train_robust_step(model_loss, t, loader, eps_scheduler, norm, train, opt, bound_type, method='robust'):
    meter = MultiAverageMeter()
    if train:
        model_loss.train()
        eps_scheduler.train()
        eps_scheduler.step_epoch()
        eps_scheduler.set_epoch_length(int((len(loader.dataset) + loader.batch_size - 1) / loader.batch_size))
    else:
        model_loss.eval()
        eps_scheduler.eval()

    for i, data in enumerate(loader):
        start = time.time()
        eps_scheduler.step_batch()
        eps = eps_scheduler.get_eps()
        xi, ti, yi = data

        # For small eps just use natural training, no need to compute LiRPA bounds
        batch_method = method
        if eps < 1e-20:
            batch_method = "natural"
        if train:
            opt.zero_grad()

        # Specify Lp norm perturbation.
        # When using Linf perturbation, we manually set element-wise bound x_L and x_U. eps is not used for Linf norm.
        if norm > 0:
            ptb = PerturbationLpNorm(norm=norm, eps=eps)
        elif norm == 0:
            ptb = PerturbationL0Norm(eps=eps_scheduler.get_max_eps(),
                                     ratio=eps_scheduler.get_eps() / eps_scheduler.get_max_eps())

        # Make the input a BoundedTensor with the pre-defined perturbation.
        x_bounded = BoundedTensor(xi, ptb)

        regular_loss = model_loss(xi, ti, yi).sum()  # regular Right Censoring
        meter.update('Loss', regular_loss.item(), xi.size(0))

        if batch_method == "robust":
            # Compute LiRPA bounds using CROWN
            lb, ub = model_loss.compute_bounds(x=(x_bounded, ti, yi), IBP=False, method="backward", bound_upper=True,
                                               bound_lower=False)
            robust_loss = ub.sum()
            loss = robust_loss
        elif batch_method == "natural":
            loss = regular_loss

        if train:
            (.1 * loss + 0.9 * regular_loss).backward()
            eps_scheduler.update_loss(loss.item() - regular_loss.item())
            opt.step()
        meter.update('Loss', loss.item(), xi.size(0))
        if batch_method != "natural":
            meter.update('Robust_Loss', robust_loss.item(), xi.size(0))

        meter.update('Time', time.time() - start)
        if i % 10 == 0 and train:
            print('[{:2d}:{:4d}]: eps={:.8f} {}'.format(t, i, eps, meter))
    print('[{:2d}:{:4d}]: eps={:.8f} {}'.format(t, i, eps, meter))

def train_robust(model,dataloader_train,dataloader_test,method,args):
    torch.manual_seed(args.seed)
    torch.cuda.manual_seed_all(args.seed)
    random.seed(args.seed)
    np.random.seed(args.seed)

    # model = BoundedModule(clf, X_train)

    ## Step 4 prepare optimizer, epsilon scheduler and learning rate scheduler
    optimizer = optim.Adam(model.parameters(), lr=args.lr)
    norm = float(args.norm)
    lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)
    eps_scheduler = eval(args.scheduler_name)(args.eps, args.scheduler_opts)

    if method == "robust":
        scheduler_opts = args.scheduler_opts
    elif method == "natural":
        scheduler_opts = f"start={args.num_epochs+1},length={args.num_epochs+1}"

    eps_scheduler = eval(args.scheduler_name)(args.eps, scheduler_opts)



    timer = 0.0
    for t in range(1, args.num_epochs+1):
        if eps_scheduler.reached_max_eps():
            # Only decay learning rate after reaching the maximum eps
            lr_scheduler.step()
        print("Epoch {}, learning rate {}".format(t, lr_scheduler.get_lr()))
        start_time = time.time()
        train_robust_step(model, t, dataloader_train, eps_scheduler, args.norm, True, optimizer, args.bound_type)
        epoch_time = time.time() - start_time
        timer += epoch_time
        print('Epoch time: {:.4f}, Total time: {:.4f}'.format(epoch_time, timer))
        print("Evaluating...")
        with torch.no_grad():
            train_robust_step(model, t, dataloader_test, eps_scheduler, norm, False, None, args.bound_type)

        if args.save_model != "":
            torch.save({'state_dict': model.state_dict(), 'epoch': t}, args.save_model)

def lower_bound(clf, nominal_input, epsilon):
    # Wrap the model with auto_LiRPA.
    model = BoundedModule(clf, nominal_input)
    # Define perturbation. Here we add Linf perturbation to input data.
    ptb = PerturbationLpNorm(norm=np.inf, eps=torch.Tensor([epsilon]))
    # Make the input a BoundedTensor with the pre-defined perturbation.
    my_input = BoundedTensor(torch.Tensor(nominal_input), ptb)
    # Regular forward propagation using BoundedTensor works as usual.
    prediction = model(my_input)
    # Compute LiRPA bounds using CROWN
    lb, ub = model.compute_bounds(x=(my_input,), method="backward")

    return lb, ub

In [ ]:
train_robust(model_robust,dataloader_train,dataloader_test,method="robust",args=args)

In [ ]:
for name,param in clf.named_parameters():
    print(name,param.grad)

In [ ]:
torch.exp(clf.k_logit(X_train))

In [ ]:
for name,param in model_robust.named_parameters():
    print(name,param)

In [ ]:
from lifelines import WeibullFitter,ExponentialFitter
clf_weib = WeibullFitter()
clf_weib.fit(durations=T_train.ravel(),event_observed=E_train.ravel())
clf_weib.plot_survival_function()

In [ ]:
from lifelines import WeibullFitter,ExponentialFitter,CoxPHFitter
clf_weib = WeibullFitter()
clf_weib.fit(durations=T_train.ravel(),event_observed=E_train.ravel())
clf_weib.plot_survival_function()

In [ ]:
X_train,T_train,E_train = dataloader_train.dataset.tensors
t = torch.linspace(0,T_train.max(),10000)

St_given_x = clf.survival_qdf(X_train,t).detach()

kmf = KaplanMeierFitter()
kmf.fit(durations=T_train,event_observed=E_train)
St_kmf  = kmf.predict(times=t.ravel().numpy())

plt.figure(figsize=(10,10))
plt.plot(t,St_kmf)
clf_weib.plot_survival_function(label="_nolegend_")
plt.plot(t,St_given_x.mean(0))
plt.ylabel("S(t)"); plt.xlabel("Time")
plt.legend(["Kaplan Meier Numerical","Weibull Fit","Neural Network"])
plt.title("Train Population Survival Curves")
plt.ylim([0,1.05])
plt.show()

In [ ]:
print(torch.exp(clf.k_logit),clf.k_logit)

In [ ]:
parameters = clf(X_train)
lam,k = parameters[:,[0]],parameters[:,[1]]
concordance_index(T_train,predicted_scores= -lam**k,event_observed=E_train)

In [ ]:
X_test,T_test,E_test = dataset_test.tensors

In [ ]:
parameters = clf(X_test)
lam,k = parameters[:,[0]],parameters[:,[1]]
concordance_index(T_test,predicted_scores= - lam**k,event_observed=E_test)

In [ ]:
clf_weib.predict(T_train.ravel()).values

lambda_,rho_ = clf_weib.params_
lambda_ = 1/lambda_

F = 1.0 - torch.exp(-(T_train*lambda_)**rho_)

concordance_index(T_train,predicted_scores=F,event_observed=E_train)

In [ ]:
clf_weib.predict(T_test.ravel()).values

lambda_,rho_ = clf_weib.params_
lambda_ = 1/lambda_

F = 1.0 - torch.exp(-(T_test*lambda_)**rho_)

concordance_index(T_test,predicted_scores=F,event_observed=E_test)

## AUTO LIRPA 

In [ ]:
import time
import random
import multiprocessing
import argparse
import torch.optim as optim
from auto_LiRPA import BoundedModule, BoundedTensor
from auto_LiRPA.perturbations import *
from auto_LiRPA.utils import MultiAverageMeter
from auto_LiRPA.eps_scheduler import LinearScheduler, AdaptiveScheduler, SmoothedScheduler, FixedScheduler
import torch.nn as nn
import torch
from auto_LiRPA import BoundedModule
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [ ]:
import torch.nn as nn

In [ ]:
clf = nn.Sequential(nn.Dropout(.5),nn.Linear(input_dims,1))

In [ ]:
class classifier(nn.Module):
    def __init__(self):
        super().__init__()

        self.dropout = nn.Dropout(0.5)
        self.linear = nn.Linear(input_dims,1)

    def forward(self,x,t,e):
        x = self.dropout(x)
        x = self.linear(x)

        return x

In [ ]:
X_train,T_train,E_train = dataset_train.tensors

In [ ]:



# class FNN(nn.Module):
#     def __init__(self):
#         super(FNN, self).__init__()
#         self.fc1 = nn.Linear(2, 4)
#         self.fc2 = nn.Linear(4, 2)
#         self.dropout = nn.Dropout(0.3)

#     def forward(self, x):
#         x = x.float()
#         x = self.fc1(x)
#         x = self.dropout(x)
#         x = self.fc2(x)
#         return x


# model_ori = FNN()
# batch_size = 100
# x = torch.normal(0, 0.2, (batch_size, 2))
# y = torch.ones((batch_size,))

# dummy_input = x[:2]
# model = BoundedModule(model_ori, dummy_input, bound_opts={
#     'relu': "same-slope", 'sparse_intermediate_bounds': False,
#     'sparse_conv_intermediate_bounds': False, 'sparse_intermediate_bounds_with_ibp': False})
# final_name1 = model.final_name

# model.train()
# c = torch.tensor([[-1, 1], [-1, 1]]).type_as(y)
# dummy_out = model(dummy_input)
# lb, ub = model(method_opt="compute_bounds", C=c, method="IBP", final_node_name=None,
#                no_replicas=True)

In [ ]:
model = BoundedModule(classifier(), (X_train,T_train,E_train))
epsilon = 1e-4
# Define perturbation. Here we add Linf perturbation to input data.
ptb = PerturbationLpNorm(norm=np.inf, eps=torch.Tensor([epsilon]))
# Make the input a BoundedTensor with the pre-defined perturbation.
my_input = BoundedTensor(torch.Tensor(X_train), ptb)
# Regular forward propagation using BoundedTensor works as usual.
# prediction = model(my_input,torch.rand_like(T_train),torch.rand_like(E_train))
# Compute LiRPA bounds using CROWN
lb, ub = model.compute_bounds( method="backward")

In [ ]:
lb, ub = model.compute_bounds(x=(my_input,T_train,E_train), method="backward")

In [ ]:
# model = BoundedModule(loss_clf, (X_train,T_train,E_train))
# # Define perturbation. Here we add Linf perturbation to input data.
# ptb = PerturbationLpNorm(norm=np.inf, eps=torch.Tensor([epsilon]))
# # Make the input a BoundedTensor with the pre-defined perturbation.
# my_input = BoundedTensor(torch.Tensor(X_train), ptb)
# # Regular forward propagation using BoundedTensor works as usual.
# # prediction = model(my_input,T_train,E_train)
# # Compute LiRPA bounds using CROWN
# lb, ub = model.compute_bounds(x=(my_input,T_train,E_train), method="backward",bound_upper=False)

In [ ]:
"""
A simple script to train certified defense using the auto_LiRPA library.

We compute output bounds under input perturbations using auto_LiRPA, and use
them to form a "robust loss" for certified defense.  Several different bound
options are supported, such as IBP, CROWN, and CROWN-IBP. This is a basic
example on MNIST and CIFAR-10 datasets with Lp (p>=0) norm perturbation. For
faster training, please see our examples with loss fusion such as
cifar_training.py and tinyimagenet_training.py
"""

import time
import random
import multiprocessing
import argparse
import torch.optim as optim
from auto_LiRPA import BoundedModule, BoundedTensor
from auto_LiRPA.perturbations import *
from auto_LiRPA.utils import MultiAverageMeter
from auto_LiRPA.eps_scheduler import LinearScheduler, AdaptiveScheduler, SmoothedScheduler, FixedScheduler
import torchvision.datasets as datasets
import torchvision.transforms as transforms



def Train(model_loss, t, loader, eps_scheduler, norm, train, opt, bound_type, method='robust'):
    meter = MultiAverageMeter()
    if train:
        model_loss.train()
        eps_scheduler.train()
        eps_scheduler.step_epoch()
        eps_scheduler.set_epoch_length(int((len(loader.dataset) + loader.batch_size - 1) / loader.batch_size))
    else:
        model_loss.eval()
        eps_scheduler.eval()

    for i, data in enumerate(loader):
        start = time.time()
        eps_scheduler.step_batch()
        eps = eps_scheduler.get_eps()
        xi,ti,yi = data

        # For small eps just use natural training, no need to compute LiRPA bounds
        batch_method = method
        if eps < 1e-20:
            batch_method = "natural"
        if train:
            opt.zero_grad()

        # Specify Lp norm perturbation.
        # When using Linf perturbation, we manually set element-wise bound x_L and x_U. eps is not used for Linf norm.
        if norm > 0:
            ptb = PerturbationLpNorm(norm=norm, eps=eps)
        elif norm == 0:
            ptb = PerturbationL0Norm(eps = eps_scheduler.get_max_eps(), ratio = eps_scheduler.get_eps()/eps_scheduler.get_max_eps())
        
        # Make the input a BoundedTensor with the pre-defined perturbation.
        x_bounded = BoundedTensor(xi, ptb)

        regular_loss = model_loss(xi,ti,yi).sum() # regular Right Censoring
        meter.update('Loss', regular_loss.item(), xi.size(0))
        if batch_method == "robust":
            # Compute LiRPA bounds using CROWN
            lb, ub = model_loss.compute_bounds(x=(x_bounded,ti,yi),IBP=False, method="backward",bound_upper=True,bound_lower=False)
            robust_loss = ub.sum()
            loss = robust_loss
        elif batch_method == "natural":
            loss =  regular_loss
        if train:
            (.1*loss+0.9*regular_loss).backward()
            eps_scheduler.update_loss(loss.item() - regular_loss.item())
            opt.step()
        meter.update('Loss', loss.item(), xi.size(0))
        if batch_method != "natural":
            meter.update('Robust_Loss', robust_loss.item(), xi.size(0))

        meter.update('Time', time.time() - start)
        if i % 10 == 0 and train:
            print('[{:2d}:{:4d}]: eps={:.8f} {}'.format(t, i, eps, meter))
    print('[{:2d}:{:4d}]: eps={:.8f} {}'.format(t, i, eps, meter))



In [ ]:
class ARGS(object):
    def __init__(self):
        pass

In [ ]:
args = ARGS()
args.verify=False
args.device="cpu"

args.seed = 123

args.eps=0.5
args.norm=np.inf
args.bound_type = "CROWN-IBP"
args.num_epochs=num_epochs
args.lr = lr
args.scheduler_name = "SmoothedScheduler"
# args.scheduler_opts = "start=5,length=10"
args.scheduler_opts = "start=5,length=10"

args.save_model = "apple.pth"


In [ ]:
clf_fragile = Exponential_Model(input_dim=input_dims,hidden_layers=hidden_dim,output_dim=output_dim)
clf_fragile.load_state_dict(clf.state_dict())

In [ ]:
# model = BoundedModule(clf, X_train)
loss_clf = RightCensorWrapper(clf)
X_train,T_train,E_train = dataloader_train.dataset.tensors
model_loss = BoundedModule(loss_clf, (X_train,T_train,E_train))


In [ ]:
torch.manual_seed(args.seed)
torch.cuda.manual_seed_all(args.seed)
random.seed(args.seed)
np.random.seed(args.seed)


## Step 4 prepare optimizer, epsilon scheduler and learning rate scheduler
opt = optim.Adam(model_loss.parameters(), lr=args.lr)
norm = float(args.norm)
lr_scheduler = optim.lr_scheduler.StepLR(opt, step_size=10, gamma=0.5)
eps_scheduler = eval(args.scheduler_name)(args.eps, args.scheduler_opts)
print("Model structure: \n", str(clf))

## Step 5: start training
if args.verify:
    eps_scheduler = FixedScheduler(args.eps)
    with torch.no_grad():
        Train(model, 1, test_data, eps_scheduler, norm, False, None, args.bound_type)
else:
    timer = 0.0
    for t in range(1, args.num_epochs+1):
        if eps_scheduler.reached_max_eps():
            # Only decay learning rate after reaching the maximum eps
            lr_scheduler.step()
        print("Epoch {}, learning rate {}".format(t, lr_scheduler.get_lr()))
        start_time = time.time()
        Train(model_loss, t, dataloader_train, eps_scheduler, norm, True, opt, args.bound_type)
        epoch_time = time.time() - start_time
        timer += epoch_time
        print('Epoch time: {:.4f}, Total time: {:.4f}'.format(epoch_time, timer))
        print("Evaluating...")
        with torch.no_grad():
            Train(model_loss, t, dataloader_test, eps_scheduler, norm, False, None, args.bound_type)
        torch.save({'state_dict': loss_clf.state_dict(), 'epoch': t}, args.save_model if args.save_model != "" else args.model)

In [ ]:
import re

In [ ]:
clf_robust = Exponential_Model(input_dim=input_dims,hidden_layers=hidden_dim,output_dim=output_dim)
clf_robust.load_state_dict({re.sub("model.","",key):value for key,value in model_loss.state_dict().items()})

In [ ]:
def lower_bound(clf,nominal_input,epsilon):
    # Wrap the model with auto_LiRPA.
    model = BoundedModule(clf, nominal_input)
    # Define perturbation. Here we add Linf perturbation to input data.
    ptb = PerturbationLpNorm(norm=np.inf, eps=torch.Tensor([epsilon]))
    # Make the input a BoundedTensor with the pre-defined perturbation.
    my_input = BoundedTensor(torch.Tensor(nominal_input), ptb)
    # Regular forward propagation using BoundedTensor works as usual.
    prediction = model(my_input)
    # Compute LiRPA bounds using CROWN
    lb, ub = model.compute_bounds(x=(my_input,), method="backward")
    
    return lb,ub

In [ ]:
plt.figure(figsize=(10,10))
X_train,T_train,E_train = dataloader_train.dataset.tensors
t = torch.linspace(0,2*T_train.max(),10000)

St_fragile_given_x = clf_fragile.survival_qdf(X_train,t).detach()
St_given_x = clf.survival_qdf(X_train,t).detach()
kmf = KaplanMeierFitter()
kmf.fit(durations=T_train,event_observed=E_train)
St_kmf = kmf.predict(times=t.ravel().numpy())


fig,axes = plt.subplots(1,2,figsize=(20,10))
axes[0].plot(t,St_kmf,linewidth=3)
axes[0].plot(t,St_fragile_given_x.mean(0),'k-',linewidth=3)
axes[0].plot(t,St_given_x.mean(0),linewidth=3)

epsilons = [0.5,0.1,0.07,0.05,0.03,0.01,0.001]
for epsilon in epsilons:
    lb,ub = lower_bound(clf_robust,X_train,epsilon)
    St_lb = torch.exp(-ub*t).mean(0)
    print("="*10 + "@ eps={}".format(epsilon) + "="*10)
    print("Train CI Unperturbed",concordance_index(event_times=T_train,predicted_scores=-clf_robust.rate_logit(X_train).detach(),event_observed=E_train))
    print("Train CI Pertubed",concordance_index(event_times=T_train,predicted_scores=-ub.detach(),event_observed=E_train))
    axes[0].plot(t,St_lb.detach(),'--')



axes[0].set_ylabel("S(t)"); axes[0].set_xlabel("Time")
axes[0].legend(["Kaplan Meier Numerical","Neural Network Nonrobust","Neural Network Robust"]+[f"LB@{epsilon}" for epsilon in epsilons])
axes[0].set_title("Robust Train Population Survival Curves")
axes[0].set_ylim([0,1])

axes[1].plot(t,St_kmf,linewidth=3)
axes[1].plot(t,St_fragile_given_x.mean(0),'k-',linewidth=3)
axes[1].plot(t,St_given_x.mean(0),linewidth=3)

print("NON ROBUST")

for epsilon in epsilons:
    lb,ub = lower_bound(clf_fragile,X_train,epsilon)
    St_lb = torch.exp(-ub*t).mean(0)
    print("="*10 + "@ eps={}".format(epsilon) + "="*10)
    print("Train CI Unperturbed",concordance_index(event_times=T_train,predicted_scores=-clf_fragile.rate_logit(X_train).detach(),event_observed=E_train))
    print("Train CI Pertubed",concordance_index(event_times=T_train,predicted_scores=-ub.detach(),event_observed=E_train))
    axes[1].plot(t,St_lb.detach(),'--')



axes[1].set_ylabel("S(t)"); axes[1].set_xlabel("Time")
axes[1].legend(["Kaplan Meier Numerical","Neural Network Nonrobust","Neural Network Robust"]+[f"LB@{epsilon}" for epsilon in epsilons])
axes[1].set_title("Nonrobust Train Population Survival Curves")
axes[1].set_ylim([0,1])

plt.show()

In [ ]:
plt.figure(figsize=(10,10))
X_test,T_test,E_test = dataloader_train.dataset.tensors
t = torch.linspace(0,2*T_test.max(),10000)

St_fragile_given_x = clf_fragile.survival_qdf(X_test,t).detach()
St_given_x = clf.survival_qdf(X_test,t).detach()
kmf = KaplanMeierFitter()
kmf.fit(durations=T_test,event_observed=E_test)
St_kmf = kmf.predict(times=t.ravel().numpy())


fig,axes = plt.subplots(1,2,figsize=(20,10))
axes[0].plot(t,St_kmf,linewidth=3)
axes[0].plot(t,St_fragile_given_x.mean(0),'k-',linewidth=3)
axes[0].plot(t,St_given_x.mean(0),linewidth=3)

epsilons = [0.5,0.1,0.07,0.05,0.03,0.01,0.001]
for epsilon in epsilons:
    lb,ub = lower_bound(clf_robust,X_test,epsilon)
    St_lb = torch.exp(-ub*t).mean(0)
    print("="*10 + "@ eps={}".format(epsilon) + "="*10)
    print("Test CI Unperturbed",concordance_index(event_times=T_test,predicted_scores=-clf_robust.rate_logit(X_test).detach(),event_observed=E_test))
    print("Test CI Pertubed",concordance_index(event_times=T_test,predicted_scores=-ub.detach(),event_observed=E_test))
    axes[0].plot(t,St_lb.detach(),'--')



axes[0].set_ylabel("S(t)"); axes[0].set_xlabel("Time")
axes[0].legend(["Kaplan Meier Numerical","Neural Network Nonrobust","Neural Network Robust"]+[f"LB@{epsilon}" for epsilon in epsilons])
axes[0].set_title("Robust Test Population Survival Curves")
axes[0].set_ylim([0,1])

axes[1].plot(t,St_kmf,linewidth=3)
axes[1].plot(t,St_fragile_given_x.mean(0),'k-',linewidth=3)
axes[1].plot(t,St_given_x.mean(0),linewidth=3)

print("NON ROBUST")

for epsilon in epsilons:
    lb,ub = lower_bound(clf_fragile,X_test,epsilon)
    St_lb = torch.exp(-ub*t).mean(0)
    print("="*10 + "@ eps={}".format(epsilon) + "="*10)
    print("Test CI Unperturbed",concordance_index(event_times=T_test,predicted_scores=-clf_fragile.rate_logit(X_test).detach(),event_observed=E_test))
    print("Test CI Pertubed",concordance_index(event_times=T_test,predicted_scores=-ub.detach(),event_observed=E_test))
    axes[1].plot(t,St_lb.detach(),'--')



axes[1].set_ylabel("S(t)"); axes[1].set_xlabel("Time")
axes[1].legend(["Kaplan Meier Numerical","Neural Network Nonrobust","Neural Network Robust"]+[f"LB@{epsilon}" for epsilon in epsilons])
axes[1].set_title("Nonrobust Test Population Survival Curves")
axes[1].set_ylim([0,1])

plt.show()

In [ ]:
lb,ub = lower_bound(clf_robust,X_test,0.1)
St_lb = torch.exp(-ub*t).detach()

In [ ]:
plt.figure(figsize=(10,10))

    
test_cases = 30

colors = list(plt.cm.brg(np.linspace(0,1,test_cases))) + ["crimson", "indigo"]

cases = np.argsort(torch.linalg.norm(St_lb - St_given_x,axis=1))[0:test_cases]
print(torch.linalg.norm(St_lb - St_given_x,axis=1)[cases])

for i,case in enumerate(tqdm(cases)):
    plt.plot(t,St_given_x[case],color=colors[i])
    plt.plot(t,St_lb[case],'--',color=colors[i])
    
plt.ylabel("S(t)"); plt.xlabel("Time")
plt.title("Individual Survival Curves Train")

In [ ]:
plt.figure(figsize=(10,10))
# lb,ub = lower_bound(model,X_train,0.1)
    
test_cases = 30
cases = torch.flip(np.argsort(torch.linalg.norm(St_lb - St_given_x,axis=1)),dims=(0,))[0:test_cases]
print(torch.linalg.norm(St_lb - St_given_x,axis=1)[cases])
for i,case in enumerate(tqdm(cases)):
    plt.plot(t,St_given_x[case],color=colors[i])
    plt.plot(t,St_lb[case],'--',color=colors[i])
    
plt.ylabel("S(t)"); plt.xlabel("Time")
plt.title("Individual Survival Curves Train")
